In [1]:
import numpy as np
import tensorflow as tf
import PIL
import cv2
from mtcnn import MTCNN

In [2]:
model = tf.keras.models.load_model("mask_detector.model")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

## TEST FACE MASK STATUS

In [9]:
# definiendo constantes
labels = ["Lleva mascarilla", "No  lleva Mascarilla"]
#IMAGE_PATH = "face_with_mask.jpg"
IMAGE_PATH = "face_normal.jpg"
#cargamos modelo
model = tf.keras.models.load_model("mask_detector.model")
#preprocesamos la imagen
image = PIL.Image.open(IMAGE_PATH)
image = image.resize((224, 224))
face = np.asarray(image)
face = tf.keras.applications.mobilenet.preprocess_input(face)
face = np.expand_dims(face, axis=0)
#realizamos la prediccion
prediction = model.predict(face)[0]
#imprimimos los resultados
label_idx = np.argmax(prediction)
label = labels[label_idx]
result = {"result":label,
        "accuracy": round(prediction[label_idx]*100,2)}
print("RESULTADO: {}, PRECISION: {}".format(result["result"], result["accuracy"]))

RESULTADO: No  lleva Mascarilla, PRECISION: 99.76


## FOR IMAGES

In [4]:
labels = ["Mask", "No Mask"]
prediction = model.predict(face)[0]
label_idx = np.argmax(prediction)
label = labels[label_idx]
print(prediction)
print(label)

[0.9963313 0.0036687]
Mask


In [7]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

labels = ["Mask", "No Mask"]

image_cv = cv2.imread("group_image4.jpg")
#image_cv = cv2.imread("group_image.jpg")

detector = MTCNN()

faces = return_position_faces(detector.detect_faces(image_cv))
labels = return_labels(faces, image_cv)
print("number of faces detected: {}".format(len(faces)))
        # Draw the rectangle around each face
for (label, (x, y, w, h)) in zip(labels,faces):
    cv2.rectangle(image_cv, (x, y), (x+w, y+h), (0, 0, 255), 2)
    cv2.putText(image_cv, label, (x, y-5), cv2.FONT_ITALIC, 0.3, (0, 0, 255), 1)
cv2.imwrite('result.png', image_cv)


C:\Users\ja_jo\AppData\Local\Temp/ipykernel_7632/1650844527.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  faces = np.array([frame[y:y+h, x:x+w] for (x, y, w, h) in faces_xy])


number of faces detected: 2


True

## FOR VIDEO

In [6]:
def return_labels(faces_xy, frame):
    labels_data = ["Mask", "No Mask"]
    faces = np.array([frame[y:y+h, x:x+w] for (x, y, w, h) in faces_xy])
    #faces = [cv2.resize(face, (224, 224)) for face in faces]
    faces = np.array([tf.keras.applications.mobilenet.preprocess_input(cv2.resize(face, (224, 224))) for face in faces])
    predictions = model.predict(faces) # array of predictions
    labels = [labels_data[np.argmax(pred)] for pred in predictions]
    return labels
def return_position_faces(list_jsons):
    # treshhold of face detected = 0.8
    return [face["box"] for face in list_jsons if face["confidence"] >0.3]

In [30]:
# Load the cascade
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
detector = MTCNN()
cv2.startWindowThread()
cap = cv2.VideoCapture(0)
salto_frames = 2
while(True): 
    # Capture frame-by-frame
    _, frame = cap.read()
    faces = return_position_faces(detector.detect_faces(frame))
    if(len(faces) > 0):
        # get labels for each face
        labels = return_labels(faces, frame)
        # Draw the rectangle around each face
        for (label, (x, y, w, h)) in zip(labels,faces):
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, label, (x, y-5), cv2.FONT_ITALIC, 0.3, (0, 0, 255), 1)
        # Display the resulting frame
    cv2.imshow("preview",frame)
    #Waits for a user input to quit the application
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# When everything done, release the capture
cv2.destroyAllWindows()